### BOJ 문제 목록 크롤링

백준 온라인 저지[https://www.acmicpc.net/](https://www.acmicpc.net/)에 존재하는 약 30,000개의 문제 정보를 크롤링하는 코드입니다.  
크롤링할 대상은 아래와 같습니다.  크롤링하기 위해서는 selenium 패키지 설치가 필요하며, `pip install selenium` 을 통해 바로 설치할 수 있습니다.

1. (번호) 문제 번호
2. (제출) 문제에 제출된 답안의 횟수
3. (정답) 정답을 맞춘 답안의 수
4. (맞힌 사람) 문제를 맞춘 사람의 수
5. (정답 비율) 문제의 정답률
6. (난이도) Solved.ac에서 제공하는 문제의 난이도

In [1]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
# 크롬드라이버 옵션 (창 안보이게 숨기기)
driver_option = webdriver.ChromeOptions()
# driver_option.add_argument('headless')

### [1] 문제 정보 크롤링 (5가지 feature)

- problemId
- problemTitle
- solved
- submissionCount
- correcRatio

In [10]:
# 크롬드라이버 실행
driver = webdriver.Chrome(options=driver_option) 
driver.implicitly_wait(3)

MAX_PAGE = 297
problem_information = []

for page in tqdm(range(MAX_PAGE)):
    driver.get(f'https://www.acmicpc.net/problemset/{page + 1}')

    # 페이지가 완전히 로딩되도록 3초동안 기다림
    time.sleep(3)

    # 문제 정보 가져오기
    problemIds       = driver.find_elements(by=By.XPATH, value="//td[@class='list_problem_id']")
    problemNames     = driver.find_elements(by=By.XPATH, value="//td[@class='list_problem_id']/following-sibling::td[1]")
    solveCounts      = driver.find_elements(by=By.XPATH, value="//td[@class='list_problem_id']/following-sibling::td[3]/a")
    submissionCounts = driver.find_elements(by=By.XPATH, value="//td[@class='list_problem_id']/following-sibling::td[4]/a")
    correctRatios    = driver.find_elements(by=By.XPATH, value="//td[@class='list_problem_id']/following-sibling::td[5]")

    list_len = {len(lst) for lst in [problemIds, problemNames, solveCounts, submissionCounts, correctRatios]}
    if len(list_len) != 1:
        print (f"[ERROR] It might have an error with page #{page}")

    for e1, e2, e3, e4, e5 in zip(problemIds, problemNames, solveCounts, submissionCounts, correctRatios):
        problem_information.append([e1.text, "\"" + e2.text + "\"", e3.text, e4.text, (e5.text)[:-1]])


driver.quit()
print(problem_information)

column_name = ['problemId', 'problemTitle', 'solved', 'submissionCount', 'correctRatio']
problem_df = pd.DataFrame(data=problem_information, columns=column_name)
problem_df.to_csv("./problems.csv", index=False)

  0%|          | 0/297 [00:00<?, ?it/s]

[['1000', '"A+B"', '268961', '974437', '39.624'], ['1001', '"A-B"', '229710', '397369', '69.902'], ['1002', '"터렛"', '35027', '204710', '22.429'], ['1003', '"피보나치 함수"', '49106', '205120', '32.818'], ['1004', '"어린 왕자"', '14541', '38627', '45.830'], ['1005', '"ACM Craft"', '12761', '69749', '28.049'], ['1006', '"습격자 초라기"', '1965', '17439', '19.931'], ['1007', '"벡터 매칭"', '3150', '11614', '37.469'], ['1008', '"A/B"', '186227', '658899', '34.586'], ['1009', '"분산처리"', '19294', '102503', '24.326'], ['1010', '"다리 놓기"', '33961', '89387', '48.080'], ['1011', '"Fly me to the Alpha Centauri"', '22342', '93733', '31.412'], ['1012', '"유기농 배추"', '45533', '169448', '37.985'], ['1013', '"Contact"', '4366', '15357', '36.692'], ['1014', '"컨닝"', '1954', '7821', '46.949'], ['1015', '"수열 정렬"', '6109', '13734', '54.554'], ['1016', '"제곱 ㄴㄴ 수"', '7980', '59192', '21.565'], ['1017', '"소수 쌍"', '2038', '10881', '25.899'], ['1018', '"체스판 다시 칠하기"', '43200', '109020', '49.721'], ['1019', '"책 페이지"', '4073', '16111', '

In [9]:
problem_df

,problemId,problemTitle,solved,submissionCount,correcRatio
0,1000,"""A+B""",268961,974437,39.624
1,1001,"""A-B""",229710,397369,69.902
2,1002,"""터렛""",35027,204710,22.429
3,1003,"""피보나치 함수""",49106,205120,32.818
4,1004,"""어린 왕자""",14541,38627,45.830
...,...,...,...,...,...
95,1095,"""마법의 구슬""",159,1454,17.805
96,1096,"""종이 접기""",69,427,30.667
97,1097,"""마법의 문자열""",295,848,47.276
98,1098,"""쌍둥이 마을""",43,506,17.917


### [2] 문제 난이도 크롤링 (2가지 feature)

- problemId
- difficulty: Bronze5(0) ~ Ruby1(29)

In [ ]:
# 크롬드라이버 실행
driver = webdriver.Chrome(options=driver_option) 
driver.implicitly_wait(3)

MAX_LEVEL = 30
difficulty_information = []

level_failed = []
page_failed = []

for level in tqdm(range(MAX_LEVEL), desc='LEVEL crawling'):
    driver.get(f'https://solved.ac/problems/level/{level + 1}')

    # 페이지가 완전히 로딩되도록 3초동안 기다림
    time.sleep(3)

    # 문제 정보 가져오기
    page_elem = driver.find_elements(by=By.CSS_SELECTOR, value='.css-18lc7iz > a')
    if len(page_elem) == 0:
        level_failed.append(f"level-{level}")
    MAX_PAGE = max([int(elem.get_attribute('href').split("=")[-1]) for elem in page_elem])

    for page in tqdm(range(MAX_PAGE), desc='Page crawling', leave=False):
        driver.get(f'https://solved.ac/problems/level/{level + 1}?page={page + 1}')

        # 페이지가 완전히 로딩되도록 3초동안 기다림
        time.sleep(3)

        # 문제 정보 가져오기
        problem_elem = driver.find_elements(by=By.CSS_SELECTOR, value='.css-lywkv4 > span > a')
        if len(problem_elem) == 0:
            page_failed.append(f"level-{level}-page-{page}")

        for elem in problem_elem:
            difficulty_information.append([int(elem.get_attribute('href').split("/")[-1]), level])

driver.quit()

column_name = ['problemId', 'difficulty']
difficulty_df = pd.DataFrame(data=difficulty_information, columns=column_name)
difficulty_df.to_csv("./difficulty.csv", index=False)

In [ ]:
print(f'- failed page list : {page_failed}')
print(f'- failed level list : {level_failed}')

In [16]:
problem_df = pd.read_csv("./problems.csv")
difficulty_df = pd.read_csv("./difficulty.csv")

difficulty_df['difficulty'] = difficulty_df['difficulty'] + 1

In [17]:
merged = problem_df.merge(difficulty_df, on='problemId', how='left')
merged = merged.sort_values(by='problemId')
merged['difficulty'] = merged['difficulty'].fillna(0)

In [18]:
merged.to_csv("./problem_information.csv", index=False)